### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [ ]:
# change the working directory to the desired location
#temporary, only for local
# import os
# os.chdir("/home/bumblebealu/groupwebsite_generator/")
#os.chdir("/Users/harshul/website clone/harshul/test/groupwebsite_generator")
#os.getcwd()

In [ ]:
#Importing classes from the Jinja2 library to load and render templates.
from jinja2 import Environment, FileSystemLoader
import json
import os
from jinja2.exceptions import UndefinedError

In [ ]:
#Function for creating proper html file names
def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

In [ ]:
#Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates
environment = Environment(loader=FileSystemLoader('templates/'))
environment.globals['page_link'] = page_link

In [ ]:
json_files = ['general', 'homepage', 'post_docs', 'undergrads', 'research_engineers',
              'faculty', 'researchers', 'grads', 'astrophysical_transients',
              'computational_metascience', 'news', 'support', 'contact']

data = {}

for json_file in json_files:
    try:
        if json_file in ["general", "homepage", "support", "contact"]:
            with open(f"../group-data/groupwebsite_data/{json_file}.json") as json_var:
                data[json_file] = json.load(json_var)

        if json_file in ["computational_metascience", "astrophysical_transients"]:
            with open(f"../group-data/groupwebsite_data/research/sub_research_data/combined/"
                      f"{json_file}.json") as json_var:
                data[json_file] = json.load(json_var)
        if json_file == "news":
            with open(f"../group-data/groupwebsite_data/news/combined_news.json") as json_var:
                data[json_file] = json.load(json_var)
        else:
            with open(f"../group-data/members/common/{json_file}.json") as json_var:
                data[json_file] = json.load(json_var)

    except (FileNotFoundError, json.JSONDecodeError):
        pass

##### Homepage

In [ ]:
homepage_template = environment.get_template('homepage.html.j2')

In [ ]:
homepage_content = \
    homepage_template.render(general=data['general'],
                             Homepage=data['homepage'],
                             computational_metascience=data['computational_metascience'
                             ],
                             astrophysical_transients=data['astrophysical_transients'
                             ])

In [ ]:
#open in write mode
with open('../kerzendorf-group.github.io/index.html', mode='w', encoding='utf-8') as Homepage:
    Homepage.write(homepage_content)

##### People Page

In [ ]:
people_template = environment.get_template("people.html.j2")

In [ ]:
required_keys = ["faculty", "post_docs", "grads", "undergrads","research_engineers", "researchers"]
if all(key in data for key in required_keys):
    try:
        people_content = people_template.render(general=data["general"], faculty=data["faculty"], post_docs=data["post_docs"], grads=data["grads"], undergrads=data["undergrads"], research_engineers=data["research_engineers"], researchers=data["researchers"])
    except UndefinedError as e:
        print(f"Error: {e.message}")
        # handle the error here
else:
    missing_key = next(key for key in required_keys if key not in data)
    print(f"Error: {missing_key} not found in data dictionary.") 

In [ ]:
with open("../kerzendorf-group.github.io/People.html", mode="w", encoding="utf-8") as people:
    people.write(people_content)

##### Individual People Page

In [ ]:
ind_person_template = environment.get_template("individual_person.html.j2")

In [ ]:
required_keys = ["faculty", "post_docs", "grads", "undergrads","research_engineers", "researchers"]
if all(key in data for key in required_keys):
    for category in [data["faculty"], data["post_docs"], data["grads"], data["undergrads"], data["research_engineers"], data["researchers"]]:
        for person in category[0:]:
            filename = f"../kerzendorf-group.github.io/members/{ page_link(person['your_directory_name'])}/{ page_link(person['your_directory_name'])}.html"
            ind_person_content = ind_person_template.render(general=data['general'], person=person)
            with open(filename, mode="w", encoding="utf-8") as page:
                page.write(ind_person_content)
else:
    missing_key = next(key for key in required_keys if key not in data)
    print(f"Error: {missing_key} not found in data dictionary.")


##### Research Page

In [ ]:
research_template = environment.get_template("research.html.j2")
sub_research_template = environment.get_template("sub_research_frontpage.html.j2")

In [ ]:
required_keys = ["astrophysical_transients", "computational_metascience"]
if all(key in data for key in required_keys):
    try:
        research_content = research_template.render(general=data["general"], computational_metascience=data["computational_metascience"], astrophysical_transients=data["astrophysical_transients"])
    except UndefinedError as e:
        print(f"Error: {e.message}")
        # handle the error here
else:
    missing_key = next(key for key in required_keys if key not in data)
    print(f"Error: {missing_key} not found in data dictionary.")

In [ ]:
with open("../kerzendorf-group.github.io/Research.html", mode="w", encoding="utf-8") as research:
        research.write(research_content)

In [ ]:
for key in required_keys:
    if key in data:
        try:
            sub_research_content = sub_research_template.render(general=data["general"], **{key: data[key]})
        except UndefinedError as e:
            print(f"Error: {e.message}")
            # handle the error here
        folder_path = f"../kerzendorf-group.github.io/sub_research/{key}"
        os.makedirs(folder_path, exist_ok=True)

        with open(f"../kerzendorf-group.github.io/sub_research/{key}.html", mode="w", encoding="utf-8") as sub_research:
            sub_research.write(sub_research_content)
    else:
        print(f"Error: {key} not found in data dictionary.")

##### News Page

In [ ]:
news_template = environment.get_template("news.html.j2")

In [ ]:
required_keys = ["news"]
if all(key in data for key in required_keys):
    try:
        news_content = news_template.render(general=data["general"],news=data["news"])
    except UndefinedError as e:
        print(f"Error: {e.message}")
        # handle the error here
else:
    missing_key = next(key for key in required_keys if key not in data)
    print(f"Error: {missing_key} not found in data dictionary.")


In [ ]:
with open("../kerzendorf-group.github.io/News.html", mode="w", encoding="utf-8") as news:
        news.write(news_content)

##### Support Page

In [ ]:
support_template = environment.get_template('support.html.j2')

In [ ]:
support_content = support_template.render(general=data["general"], support=data["support"])

In [ ]:
with open('../kerzendorf-group.github.io/Support.html', mode='w', encoding='utf-8') as support:
    support.write(support_content)

##### Contact

In [ ]:
contact_template = environment.get_template('contact.html.j2')

In [ ]:
contact_content = contact_template.render(general=data["general"], contact=data["contact"])

In [ ]:
with open('../kerzendorf-group.github.io/Contact.html', mode='w', encoding='utf-8') as contact:
    contact.write(contact_content)